# M203 Electronic markets project

## Exercise 2 - Optimal POV

Marchessaux François, Collin Thibault

### Loading libraries and initializing parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# Initialize the fixed parameters for inventory, risk aversion, market impact and volatility
x0 = 10e4
sigma = 0.20
T = 1
fixed_eta = 1.1 * 1e-4
fixed_lamb = 0.1

# Initialize the varying parameters
t_values = np.linspace(0, T, 1000)
lambdas = [85e-4, 1e-1 , 0.5, 1, 3, 8]
lbd_values = np.linspace(1e-4, 1, num=1000)
etas = [1e-6, 1e-5, 1e-4, 5e-4, 1e-3]

### Percentage of Volume - optimal strategy

In [ ]:
def percentage_of_volume_opti_strat(t, x0, sigma, lamb, eta):
    """
    Computes POV optimal deterministic strategy at a given time and for given parameters

    t: current time
    x0: initial inventory
    sigma: underlying spot volatility
    lamb: risk aversion parameter
    eta: multiplicative factor for temporary market impact
    """

    v = np.sqrt((lamb * (x0 ** 2) * (sigma ** 2)) / (3 * eta))
    return x0 - v * t

#### Varying risk aversion

In [ ]:
df = pd.DataFrame()

# Compute the optimal strategy for each level of risk aversion
for lamb in lambdas:
    xt = [max(percentage_of_volume_opti_strat(t, x0, sigma, lamb, fixed_eta), 0) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'lambda': str(lamb)})
    df = pd.concat([df, temp_df], ignore_index=True)

# Display the strategies
fig = px.line(df, x='t', y='x', color='lambda', template='plotly_white',
              labels={"x": "Remaining Inventory", "t": "Time", "lambda": "Risk Aversion"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

#### Varying temporary market impact

In [ ]:
df = pd.DataFrame()

# Compute the optimal strategy for each level of temporary market impact
for eta in etas:
    xt = [max(percentage_of_volume_opti_strat(t, x0, sigma, fixed_lamb, eta), 0) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'eta': str(eta)})
    df = pd.concat([df, temp_df], ignore_index=True)

# Display the strategies
fig = px.line(df, x='t', y='x', color='eta', template='plotly_white',
              labels={"x": "Remaining Inventory", "t": "Time", "eta": "Market Impact"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

### Percentage of volume - efficient frontier

In [ ]:
def calculate_POV_expectation(lbd_values, eta, sigma, x0):
    """
    Computes the expectation for the PoV order, given the closed-form solved formula

    lbd_values: different levels of risk aversion
    eta: multiplicative factor for temporary market impact
    sigma: underlying spot volatility
    x0: initial inventory
    """

    v = np.sqrt((lbd_values * (x0 ** 2) * (sigma ** 2)) / (3 * eta))
    return -eta * x0 * v

In [ ]:
def calculate_POV_variance(lbd_values, eta, sigma, x0):
    """
    Computes the variance for the PoV order, given the closed-form solved formula

    lbd_values: different levels of risk aversion
    eta: multiplicative factor for temporary market impact
    sigma: underlying spot volatility
    x0: initial inventory
    """
    v = np.sqrt((lbd_values * (x0 ** 2) * (sigma ** 2)) / (3 * eta))
    return (x0 ** 3) * (sigma ** 2) / (3 * v)

In [ ]:
# Compute the variance and the expectation for different level of risk aversion
expectation_POV = calculate_POV_expectation(lbd_values, fixed_eta, sigma, x0)
variance_POV = calculate_POV_variance(lbd_values, fixed_eta, sigma, x0)

# Store and scale the results
df = pd.DataFrame({'E': np.abs(expectation_POV) / 100, 'V': np.abs(variance_POV) / 10000, 'lambda': lbd_values})

# Display the efficient frontier
fig = px.scatter(df, x='V', y='E', color='lambda', template='plotly_white',
                 labels={"V": "Variance", "E": "Abs(Expectation)", "lambda": "𝛌"}, color_continuous_scale='Purp')
fig.update_layout(width=900, height=300)
fig.update_layout(xaxis=dict(range=[800, 14000]))
fig.show()

In [ ]:
def calculate_IS_expectation(k, eta, x0, T):
    """
    Computes the expectation for the implementation shortfall order, given the closed-form solved formula

    eta: multiplicative factor for temporary market impact
    x0: initial inventory
    T: end date of the execution process
    """
    return -((((k * x0) / (np.sinh(k * T)))**2) * eta) * ((2 * k * T + np.sinh(2 * k * T)) / (4 * k))

In [ ]:
def calculate_IS_variance(k, sigma, x0, T):
    """
    Computes the variance for the implementation shortfall order, given the closed-form solved formula

    sigma: underlying spot volatility
    x0: initial inventory
    T: end date of the execution process
    """
    return (((sigma * x0) / (np.sinh(k * T)))**2) * ((np.sinh(2 * k * T) - (2 * k * T)) / (4 * k))

In [ ]:
# Compute the variance and the expectation of IS orders
k_values = np.sqrt(lbd_values / fixed_eta) * sigma
expectation_IS = calculate_IS_expectation(k_values, fixed_eta, x0, T)
variance_IS = calculate_IS_variance(k_values, sigma, x0, T)

# Compute the difference between the two types of orders to compare them
difference_expectation = np.abs(expectation_POV - expectation_IS) / 100
difference_variance = np.abs(variance_POV - variance_IS) / 10000

df_difference = pd.DataFrame({'Difference_E': difference_expectation, 'Difference_V': difference_variance, 'lambda': lbd_values})

# Display the result
fig = px.scatter(df_difference, x='Difference_V', y='Difference_E', color='lambda', template='plotly_white',
                 labels={"Difference_V": "Variance Difference", "Difference_E": "Abs(Expectation Difference)", "lambda": "𝛌"}, color_continuous_scale='Blugrn')

fig.update_layout(width=900, height=300)
fig.update_layout(xaxis=dict(range=[0, 5000]))
fig.show()